In [11]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection

processor = AutoImageProcessor.from_pretrained("PekingU/rtdetr_r101vd", cache_dir="./hf-models")
model = AutoModelForObjectDetection.from_pretrained("PekingU/rtdetr_r101vd", cache_dir="./hf-models")
model.to("cuda")

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

In [16]:
import torch
from PIL import Image, ImageDraw
import requests



image = Image.open('./images/eva.jpg')


inputs = processor(images=image, return_tensors="pt").to("cuda")


model.eval()
outputs = model(**inputs)


target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)


draw = ImageDraw.Draw(image)
for result in results:
    for score, label_id, box in zip(result["scores"], result["labels"], result["boxes"]):
        score, label = score.item(), label_id.item()
        box = [round(i, 2) for i in box.tolist()]
        draw.rectangle(box, outline="red", width=3)
        label_text = model.config.id2label[label]
        draw.text((box[0], box[1]), f"{label_text}: {score:.2f}", fill="red")

# image

In [6]:
inputs.keys()

dict_keys(['pixel_values'])

In [15]:
outputs[0].to("cpu")

tensor([[[-3.0867, -4.8614, -4.4898,  ..., -3.7823, -5.7635, -4.5013],
         [-1.9239, -3.9590, -4.6265,  ..., -3.9624, -5.0224, -4.1024],
         [-0.1306, -4.0596, -3.6781,  ..., -4.8141, -5.2083, -5.0609],
         ...,
         [-4.5015, -6.0017, -5.1284,  ..., -5.9943, -7.6003, -5.8957],
         [-3.4615, -5.6207, -5.6251,  ..., -5.3165, -6.5232, -6.2966],
         [-4.3224, -4.8673, -4.8007,  ..., -4.5635, -6.6574, -4.8908]]],
       grad_fn=<ToCopyBackward0>)

In [1]:
import cv2
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
ret, frame = cap.read()
print(ret, frame.shape)

cv2.imshow("Network Camera", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

True (720, 1280, 3)
